# Cyberbullying Classification

### 1. The degree of aggressivity & Negativity

### 2. Personnalization
    - Depency tree, establish the subject, object, complement, aux
    - Also, tell the (person 1st, 2nd, 3rd) of the subject and the object. 
    - Identify what are they accuse from 
    - Identify who’s being targeted (Lexicon or wordnet)
    - Established if the aux is directed to the object  (Establishing intention)
### 3. Harms inflicted and to who
    - Now, using the complement (output of dependency tree), we can map these to threats
    - We need wordnet to identify threat. (Actually better lexicon than word net)
    - To object refers to what category? race, nationality, religion, color, gender, 



Set your environment
- source spacy/bin/activate (Bash)
- get back to your base: deactivate (Bash)

You have to be in anaconda3 interpreter

# Import data

In [2]:
import numpy as np
import pandas as pd 
import os
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
import re
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from spacy import displacy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import spacy
import nltk
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()


/Users/philippebeliveau/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
macbook = ('/Users/philippebeliveau/Desktop/Notebook/Winter_2024/Text_mining/Git_MiningRepository/Text_miningProject/Notebook/cyberbullying_tweets.csv')

mac_mini = ('/Users/philippebeliveau/Desktop/Notebook_Jupyter_R/Winter_2024/Text_mining/Project/Text_miningProject/Notebook/cyberbullying_tweets.csv')

df = pd.read_csv(macbook)
df.head()
categories = df['cyberbullying_type'].unique()


In [4]:
categories

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [5]:
# Initialize an empty dictionary to hold the tweets by category
tweets_by_category = {}

# Iterate over each category
for category in categories:
    # Filter the dataset for the current category
    category_tweets = df[df['cyberbullying_type'] == category]['tweet_text'].tolist()
    
    # Add the list of tweets to the dictionary
    tweets_by_category[category] = category_tweets

category = tweets_by_category['ethnicity'][100:120]

# Data preprocessing

In [6]:
# Filter 2000 tweets from each category
df = df.groupby('cyberbullying_type').apply(lambda x: x.sample(min(len(x), 2000))).reset_index(drop=True)

# Now you can work with 'sampled_df' which contains 2000 samples from each category
df.shape[0]

12000

In [7]:
import re

def clean_text(text):
    #removing hastags and links
    pattern=re.compile(r"(#[a-zA-Z0-9]+|@[a-zA-Z0-9]+|https?://\S+|www\.\S+|\S+\.[a-z]+|RT @)")
    text = pattern.sub('', text)
    text = " ".join(text.split())
    
    return text

In [8]:
#cleaning of data
df['clean_data']=df['tweet_text'].apply(lambda x: clean_text(x))

In [9]:
df.shape[0]

12000

In [10]:
# Initialize an empty dictionary to hold the tweets by category
tweets_by_category = {}

# Iterate over each category
for category in categories:
    # Filter the dataset for the current category
    category_tweets = df[df['cyberbullying_type'] == category]['clean_data'].tolist()
    
    # Add the list of tweets to the dictionary
    tweets_by_category[category] = category_tweets

category = tweets_by_category['ethnicity'][100:120]

# Part 2 - Critique vs Insult

# Simple approach
- Extract the adjectives, nouns, the Subject Person, and the complement/conjunction. 
- Extract the name entities
- Extract all the synonyms of the adjective, nouns and complement into a long list.
- Create a polarity score using Vadersentiment on the tweet itself and on the list of synonyms

In [11]:
Gender = ["rape is real..zvasiyana nema jokes about being drunk or being gay or being lesbian...rape is not ones choice or wish..thtz where the sensitivity is coming from",
         "You never saw any celebrity say anything like this for Obama: B Maher Incest Rape 'Joke' S Colbert Gay 'joke' K Griffin beheading 'joke'",
         "@ManhattaKnight I mean he's gay, but he uses gendered slurs and makes rape jokes",
         "RT @Raul_Novoa16: @AliciaBernardez @Alex_Aim @_mecaesmal feminazi",
         "Rape is rape. And the fact that I read one post about a guy getting raped and the comments are calling him gay and he should be happy...? stfu and I really hope no one takes this as a joke tf you own no ones body. You have no rights to do whatever you want to someone else.",
         "@coiny Also, it's hard to take a company seriously re: harassment when the game has rape jokes & gay jokes and treats women the way it does.",
         "Idgaf if you are gay, lesbian, bisexual, or whatever the fuck you are, you don't fuck make a rape joke and think it's fucken ok",
         "#GermanProfessor gives meaning to term FemiNazi. Doesn't hire 'Indian Men' bcz of 'Indias's Rape Problem'",
         "RT @mcclure111: #DontDateSJWs #ThatWouldBeAViolationOfTheRestrainingOrder",
         "So I call you female I’m basically calling you a bitch ?"]

Ethnicity = [
"Hey dumb fuck celebs stop doing something for people for publicity on Facebook... Wtf happen to life u niggers are cowards.",
"Fuck u bitch RT @tayyoung_: FUCK OBAMA, dumb ass nigger",
"@JoeBiden No Joe, YOU are the RACIST. They have so many video and audio clips of the RACIST shit you have spewed in your past, How can anyone see you as anything but a RACIST. You hate Black people just admit it. And you loves the women to touch. Just look at your own stuff!",
"when your truck looks dumb as fuck out trying to make it look worse. you ain't country. quit tryin nigger.",
"That nigger food in the cafe today was disgusting",
"Muthafuckas still EGO tripping!Fool fuck your damn ego and live life to the fullest!!! LOL at these dumb ass niggers with the bullshit!",
"@iMADE_YouMOAN don't play dumb, I should fuck yo little white ass up you nigger. 3chainz.? Aw.",
"I WILL ONLY SAY IT ONCE! IF YOU DO NOT FEAR HAVE THE SUSPICION OF BEING CALLED A NIGGER! NIGGA! OR NEGRO AT ANY POINT IN YOUR LIFE YOU CANNOT USE THE N WORD. PERIOD. I am talking about non black people in specific!!!! White passing mixed kids are on THIN ICE.",
"@CJ_Jacobs oh look it's that dumb niggers ghost acct! Fuck you you dumb motherfucker!",
"@Otoliver @Yo_PaTnA_dEm20 this bitch is dumb talk bout 'do not dew 'said not sayed' nigger not nigga lol bitch fuck off"
]

noncyber = [
"In other words #katandandre, your food was crapilicious! #mkr",
"Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc",
"@XochitlSuckkks a classy whore? Or more red velvet cupcakes?",
"@Jason_Gio meh. :P  thanks for the heads up, but not too concerned about another angry dude on twitter.",
"@RudhoeEnglish This is an ISIS account pretending to be a Kurdish account.  Like Islam, it is all lies.",
"@Raja5aab @Quickieleaks Yes, the test of god is that good or bad or indifferent or weird or whatever, it all proves gods existence.",
"Itu sekolah ya bukan tempat bully! Ga jauh kaya neraka",
"Karma. I hope it bites Kat on the butt. She is just nasty. #mkr",
"@stockputout everything but mostly my priest",
"Rebecca Black Drops Out of School Due to Bullying:"
]


In [12]:
# Initialize lists to store data

# Create a VADER sentiment analyzer object
analyzer = SentimentIntensityAnalyzer()
data = []

for tweet in df['clean_data']:
    # Analyze the tweet
    doc = nlp(tweet)

    # Extract adjectives, nouns, subjects and complements/conjunctions/root
    adjectives = [token.text for token in doc if token.pos_ == "ADJ"]
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    subjects = [(token.text, token.morph.get('Person')) for token in doc if token.dep_ == "nsubj"]
    complements = [token.text for token in doc if token.dep_ in ("acomp", "conj", "ROOT")]

    # Extract children of ccomp or conj
    ccomp_conj_children = []
    ccomp_conj_sentiments = []
    for token in doc:
        if token.dep_ in ("ccomp", "conj", 'ROOT'):
            children = [child.text for child in token.children]
            ccomp_conj_children.append(children)
            # Compute sentiment scores for each child and append to list
            ccomp_conj_sentiments.extend([analyzer.polarity_scores(child)["compound"] for child in children])

    # Compute sum of sentiment scores for ccomp or conj children
    sum_ccomp_conj_sentiment = sum(ccomp_conj_sentiments)

    # Extract named entities and their types
    named_entities = [(ent.text, ent.label_) for ent in doc.ents]

    # Extract synonyms
    synonyms = []
    for token in doc:
        if token.pos_ in ("ADJ", "NOUN", 'ccomp'):
            for syn in wordnet.synsets(token.text):
                for lemma in syn.lemmas():
                    synonyms.append(lemma.name())

    # Create a polarity score
    sentiment_score = analyzer.polarity_scores(tweet)["compound"]
    synonyms_sentiment_score = analyzer.polarity_scores(" ".join(synonyms))["compound"]

    # Append data to list
    data.append({
        "Tweet": tweet,
        "Adjectives": adjectives,
        "Nouns": nouns,
        "Subjects": subjects,
        "Complements": complements,
        "CCOMP or CONJ Children": ccomp_conj_children,
        "CCOMP or CONJ Children Sum Sentiment": sum_ccomp_conj_sentiment,
        "Named Entities": named_entities,
       # "Synonyms": synonyms,
        "Sentiment Score": sentiment_score,
        "Synonyms Sentiment Score": synonyms_sentiment_score
    })

# Create DataFrame
df = pd.DataFrame(data)

KeyboardInterrupt: 

In [ ]:
# Test with new elements
from nltk.sentiment.util import mark_negation
from nltk.corpus import sentiwordnet as swn

# Initialize lists to store data
analyzer = SentimentIntensityAnalyzer()
data = []

for tweet in df['clean_data']:
    # Analyze the tweet
    doc = nlp(tweet)

    # Extract adjectives, nouns, subjects and complements/conjunctions/root
    adjectives = [token.text for token in doc if token.pos_ == "ADJ"]
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    subjects = [(token.text, token.morph.get('Person')) for token in doc if token.dep_ == "nsubj"]
    complements = [token.text for token in doc if token.dep_ in ("acomp", "conj", "ROOT")]

    # Extract children of ccomp or conj
    ccomp_conj_children = []
    ccomp_conj_sentiments = []
    for token in doc:
        if token.dep_ in ("ccomp", "conj", 'ROOT'):
            children = [child.text for child in token.children]
            ccomp_conj_children.append(children)
            # Compute sentiment scores for each child and append to list
            ccomp_conj_sentiments.extend([analyzer.polarity_scores(child)["compound"] for child in children])

    # Compute sum of sentiment scores for ccomp or conj children
    sum_ccomp_conj_sentiment = sum(ccomp_conj_sentiments)

    # Extract named entities and their types
    named_entities = [(ent.text, ent.label_) for ent in doc.ents]

    # Extract synonyms
    synonyms = []
    for token in doc:
        if token.pos_ in ("ADJ", "NOUN", 'ccomp'):
            for syn in wordnet.synsets(token.text):
                for lemma in syn.lemmas():
                    synonyms.append(lemma.name())

    # Create a polarity score
    sentiment_score = analyzer.polarity_scores(tweet)["compound"]
    synonyms_sentiment_score = analyzer.polarity_scores(" ".join(synonyms))["compound"]

    # Check if "?" is in the tweet
    has_question_mark = "?" in tweet

    # Mark negation in the tweet
    negated_tweet = " ".join(mark_negation(tweet.split()))

    # Check for discourse markers
    discourse_markers = ['say', 'claim']
    has_discourse_marker = any(marker in tweet for marker in discourse_markers)

    # Check if there is a 3rd person verb in the tweet
    has_third_person_verb = any(token.morph.get('Person') == '3' and token.pos_ == 'VERB' for token in doc)

    # Get SentiWordNet polarity scores
    senti_scores = []
    for token in doc:
        synsets = list(swn.senti_synsets(token.text))
        if synsets:
            senti_scores.append(synsets[0].pos_score() - synsets[0].neg_score())

    avg_senti_score = sum(senti_scores) / len(senti_scores) if senti_scores else 0

    # Append data to list
    data.append({
        "Tweet": tweet,
        "Has Question Mark": has_question_mark,
        "Negated Tweet": negated_tweet,
        "Has Discourse Marker": has_discourse_marker,
        "Tweet": tweet,
        "Adjectives": adjectives,
        "Nouns": nouns,
        "Subjects": subjects,
        "Complements": complements,
        "CCOMP or CONJ Children": ccomp_conj_children,
        "CCOMP or CONJ Children Sum Sentiment": sum_ccomp_conj_sentiment,
        "Named Entities": named_entities,
       # "Synonyms": synonyms,
        "Sentiment Score": sentiment_score,
        "Synonyms Sentiment Score": synonyms_sentiment_score,  
        "Has Third Person Verb": has_third_person_verb,
        "Average SentiWordNet Score": avg_senti_score,
    })

# Create DataFrame
df = pd.DataFrame(data)

In [ ]:
df.head()

,Tweet,Has Question Mark,Negated Tweet,Has Discourse Marker,Adjectives,Nouns,Subjects,Complements,CCOMP or CONJ Children,CCOMP or CONJ Children Sum Sentiment,Named Entities,Sentiment Score,Synonyms Sentiment Score,Has Third Person Verb,Average SentiWordNet Score
0,I’m sending this comment to my daughter. Six months off school and the bullies are still slagging her off when they haven’t heard of or seen her in all that time!,False,I’m sending this comment to my daughter. Six months off school and the bullies are still slagging her off when they haven’t heard of or seen her in all that time!,False,[],"[comment, daughter, months, school, bullies, time]","[(I, [1]), (bullies, []), (they, [3])]","[sending, slagging, seen]","[[I, ’m, comment, to, .], [off, bullies, are, still, her, off, heard, !], [her, in]]",0.0000,"[(Six months, DATE)]",0.0000,-0.9451,False,-0.112500
1,This gives me the same vibe as when the girl who constantly bullied the fuck out of me in high school tried to message me 5 years later like “long time no talk omg I miss you!! Have you ever heard of ITWORKS-“,False,This gives me the same vibe as when the girl who constantly bullied the fuck out of me in high school tried to message me 5 years later like “long time no talk_NEG omg_NEG I_NEG miss_NEG you!!_NEG Have_NEG you_NEG ever_NEG heard_NEG of_NEG ITWORKS-“_NEG,False,"[same, high, long]","[vibe, girl, fuck, school, years, time, talk, omg]","[(This, []), (girl, []), (who, []), (I, [1]), (you, [2])]","[miss, heard]","[[This, me, vibe], [gives, omg, I, you, !, !], [Have, you, ever, of, “]]",0.0000,"[(5 years later, DATE)]",-0.8585,-0.6942,False,-0.008065
2,Hilarious how the girl who bullied me in high school and called me fat every single day of my life and told me I could go on a diet but I would never be pretty and that I should k*ll myself because I’m fat is on Instagram saying how she “totally understands eating disorders”,False,Hilarious how the girl who bullied me in high school and called me fat every single day of my life and told me I could go on a diet but I would never be_NEG pretty_NEG and_NEG that_NEG I_NEG should_NEG k*ll_NEG myself_NEG because_NEG I’m_NEG fat_NEG is_NEG on_NEG Instagram_NEG saying_NEG how_NEG she_NEG “totally_NEG understands_NEG eating_NEG disorders”_NEG,True,"[high, fat, single, pretty, fat]","[girl, school, day, life, diet, disorders]","[(who, []), (I, [1]), (I, [1]), (I, [1]), (I, [1]), (she, [3])]","[Hilarious, called, told, be, pretty, k*ll, fat, is]","[[girl], [me, fat, day, and, told], [me, go], [I, could, on], [but, I, would, never, pretty, and, k*ll, is, ”], [that, I, should, myself, ’m], [on, saying], [how, she, “, totally, disorders]]",0.4939,[],-0.6300,0.7096,False,0.027027
3,Hahaha this video reminds me of some shitty elementary school bully fuck these people,False,Hahaha this video reminds me of some shitty elementary school bully fuck these people,False,"[shitty, elementary]","[video, bully, people]","[(Hahaha, []), (video, [])]",[reminds],"[[Hahaha, video, me, of, fuck]]",0.0151,"[(Hahaha, PERSON)]",-0.7717,-0.9790,False,-0.187500
4,"I don't care if the chance of children dying is virtually 0. We must protect our children from virus', bullies, and gender classifications. Special schools need to be built to protect our fragile babies.",False,"I don't care_NEG if_NEG the_NEG chance_NEG of_NEG children_NEG dying_NEG is_NEG virtually_NEG 0._NEG We_NEG must_NEG protect_NEG our_NEG children_NEG from_NEG virus',_NEG bullies,_NEG and_NEG gender_NEG classifications._NEG Special_NEG schools_NEG need_NEG to_NEG be_NEG built_NEG to_NEG protect_NEG our_NEG fragile_NEG babies._NEG",False,"[Special, fragile]","[chance, children, children, virus, bullies, gender, classifications, schools, babies]","[(I, [1]), (chance, []), (We, [1]), (schools, [])]","[care, protect, bullies, classifications, need]","[[I, do, n't, is, .], [We, must, children, from, ', ,, bullies, .], [,, and, classifications], [gender], [schools, built, .]]",0.0000,[]

In [84]:
def contains_person(subjects, person):
    person_list = [p for _, p_list in subjects for p in p_list]
    return str(person) in person_list

df['Has 1st Person'] = df['Subjects'].apply(lambda x: contains_person(x, 1))
df['Has 2nd Person'] = df['Subjects'].apply(lambda x: contains_person(x, 2))
df['Has 3rd Person'] = df['Subjects'].apply(lambda x: contains_person(x, 3))

df['Word Count'] = df['Tweet'].apply(lambda x: len(x.split()))
df['Adjusted Sentiment Score'] = df.apply(lambda row: row['CCOMP or CONJ Children Sum Sentiment'] / len(row['CCOMP or CONJ Children']) if len(row['CCOMP or CONJ Children']) != 0 else 0, axis=1)
df['Has Capital Word'] = df['Tweet'].apply(lambda x: any(word.isupper() for word in x.split()))


### Save df

In [85]:
# Export csv
df.to_csv('/Users/philippebeliveau/Library/Mobile Documents/com~apple~CloudDocs/_Bureau_/Master/Winter_2024/Text_mining/Project/Dataframe/smaller_sample.csv')

In [86]:
# Import csv 
df1 = pd.read_csv('/Users/philippebeliveau/Library/Mobile Documents/com~apple~CloudDocs/_Bureau_/Master/Winter_2024/Text_mining/Project/Dataframe/smaller_sample.csv')

# Classification

In [96]:
def classify(row):
    education_terms = ['school', 'schools', 'college', 'graduation']
    if any(term in str(word) for term in education_terms for word in row[['Nouns', 'Tweet']]):
        return 'Critique/Other'
    elif row['Adjusted Sentiment Score'] > -0.20:
        return 'Critique/Other'
    elif row['Adjusted Sentiment Score'] < -0.15 and row['Sentiment Score'] > 0.3 and row['Synonyms Sentiment Score'] > 0.2:
        return 'Critique/Other'
    elif row['Has Capital Word'] == True and row['Adjusted Sentiment Score'] < -0.10 and 'you' in row['Subjects'] or 'your' in row['Subjects']:
        return 'Insults'
    else:
        return 'Insults'

df1['Classification'] = df1.apply(classify, axis=1)
df1.shape[0]
df1.head()

,Unnamed: 0,Tweet,Adjectives,Nouns,Subjects,Complements,CCOMP or CONJ Children,CCOMP or CONJ Children Sum Sentiment,Named Entities,Sentiment Score,Synonyms Sentiment Score,Has 1st Person,Has 2nd Person,Has 3rd Person,Word Count,Adjusted Sentiment Score,Has Capital Word,Classification
0,0,"I just saw a girl who bullied me in high school twerking on facebook live, good for her lmao","['high', 'good']","['girl', 'school', 'twerking', 'facebook', 'lmao']","[('I', ['1']), ('who', [])]",['saw'],"[['I', 'just', 'girl']]",0.0000,[],0.4019,0.9986,True,False,False,19,0.00000,True,Critique/Other
1,1,the girls who wore this shit bullied me in high school so if that gives any context to its audience,['high'],"['girls', 'shit', 'school', 'context', 'audience']","[('girls', []), ('who', []), ('that', [])]",['bullied'],"[['girls', 'me', 'in', 'so', 'gives']]",0.0000,[],-0.8537,-0.9974,False,False,False,20,0.00000,False,Critique/Other
2,2,i actually feel sick i don’t understand how it’s men > women rn omg i don’t wanna go back to that school they’re gonna bully me and make fun of me for being bi,['sick'],"['men', 'women', 'omg', 'school', 'fun', 'bi']","[('i', ['1']), ('i', ['1']), ('it', ['3']), ('i', ['1']), ('they', ['3'])]","['feel', 'sick', 'make']","[['i', 'actually', 'sick', 'understand', 'wanna', 'gon', 'bi'], ['i', 'do', 'n’t', '’s'], ['how', 'it', 'men', '>', 'rn'], ['they', '’re', 'bully', 'and', 'make'], ['fun', 'of', 'for']]",-0.4939,[],-0.4939,-0.9814,True,False,True,34,-0.09878,False,Critique/Other
3,3,"you’d have to get girls who bullied in high school to ADMIT they were bullies first, though..... imagine lmao",['high'],"['girls', 'school', 'bullies']","[('you', ['2']), ('who', []), ('they', ['3'])]",['have'],"[['you', '’d', 'get', 'imagine'], ['they', 'bullies', 'first', ',']]",0.0000,"[('first', 'ORDINAL')]",0.3254,-0.9260,False,True,True,19,0.00000,True,Critique/Other
4,4,"The God of Abraham works this way: dirt done is visited upon perpetrators. Your face is in headlines news across Europe for lowlife abusive language. Nothing to do representing Christian decency of Floridians, just making America look like an immature school yard of bullies.","['abusive', 'Christian', 'immature']","['way', 'dirt', 'perpetrators', 'face', 'headlines', 'news', 'lowlife', 'language', 'decency', 'school', 'yard', 'bullies']","[('God', []), ('face', []), ('America', [])]","['visited', 'is', 'Nothing']","[['God', 'way'], ['works', ':', 'dirt', 'is', 'upon', '.'], ['face', 'in', '.'], ['do', ',', 'making', '.'], ['America', 'like']]",0.2944,"[('Abraham', 'PERSON'), ('Europe', 'LOC'), ('Christian', 'NORP'), ('Floridians', 'NORP'), ('America', 'GPE')]",-0.7579,-0.9952,False,False,False,44,0.05888,False,Critique/Other


# Evaluation

In [106]:
import pandas as pd

# Set max rows and columns
# Set max column width
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# Now print your dataframe
df1[['Tweet', 'Classification', 'Adjusted Sentiment Score', 'Subjects','Complements', 
     "CCOMP or CONJ Children",'Has 1st Person', 'Has 2nd Person', 'Has 3rd Person', 'Named Entities']].sample(10)


,Tweet,Classification,Adjusted Sentiment Score,Subjects,Complements,CCOMP or CONJ Children,Has 1st Person,Has 2nd Person,Has 3rd Person,Named Entities
7971,So you admit you were wrong about Muslim extremists?,Insults,-0.238350,"[('you', ['2']), ('you', ['2'])]","['admit', 'wrong']","[['So', 'you', 'were', '?'], ['you', 'wrong']]",False,True,False,"[('Muslim', 'NORP')]"
9064,Stop trolling you idiot !!!,Insults,-0.296000,"[('you', ['2'])]",['idiot'],"[['Stop', 'you', '!', '!', '!']]",False,True,False,[]
222,folding under corporate pressure after allowing me to be bullied by Blue Check Journalist who are high off jealousy and running a smear campaign,Critique/Other,0.000000,"[('who', [])]","['folding', 'running']","[['under', 'after'], ['me', 'to', 'be', 'by'], ['who', 'off', 'and', 'running'], ['campaign']]",False,False,False,"[('Blue Check', 'ORG')]"
4677,"What substance is being expressed when they attack women with vile sexist slurs, gay men with vile homophobic slurs, black people with vile racist slurs. When they threaten peoples' children. When they threaten women with rape.",Critique/Other,0.000000,"[('they', ['3']), ('they', ['3']), ('they', ['3'])]","['expressed', 'people', 'threaten', 'threaten']","[['substance', 'is', 'being', 'men', '.'], ['black', 'with'], ['When', 'they', 'children', '.'], ['When', 'they', 'women', 'with', '.']]",False,False,True,[]
4362,"_ dis s more lyk a gay joke, not rape! lol",Critique/Other,0.000000,"[('dis', [])]","['_', 'rape', 'lol']","[[], ['dis', '!'], []]",False,False,False,"[('dis s', 'PERSON')]"
8621,"first, i noticed the misogyny. but then i realized - plz brush your teeth.",Critique/Other,0.025733,"[('i', ['1']), ('i', ['1'])]","['noticed', 'realized', 'plz']","[['first', ',', 'i', 'misogyny', '.'], ['but', 'then', 'i', 'plz', 'brush', '.'], ['teeth']]",True,False,False,"[('first', 'ORDINAL')]"
11090,"This piece reads like a bizarro PSA from the 50s: ""'far right ideology bled into his daily life. He began referring to himself as a conservative. He dated a christian woman.' 'it was kind of sad. How did you get this way?' Said, a friend of Mr. Cain""",Critique/Other,0.063067,"[('piece', []), ('ideology', []), ('He', ['3']), ('He', ['3']), ('it', ['3']), ('you', ['2'])]","['reads', 'began', 'dated', 'was', 'sad', 'get', 'Said']","[['piece', 'like', ':', 'bled', '.'], ['He', 'referring', '.'], ['He', 'woman', '.', ""'""], [""'"", 'it', 'sad', '.'], ['How', 'did', 'you', 'way', '?', ""'""], [',', 'friend', '""']]",False,True,True,"[('PSA', 'ORG'), ('50s', 'DATE'), ('christian', 'NORP'), ('Said', 'PERSON'), ('Cain', 'PERSON')]"
5456,I wanna feel bad for you but then i remember you made rape jokes/anti gay jokes sooo,Critique/Other,-0.180767,"[('I', ['1']), ('i', ['1']), ('you', ['2'])]","['feel', 'bad', 'remember']","[['I', 'wanna', 'bad', 'for', 'but', 'remember', 'sooo'], ['then', 'i', 'made'], ['you']]",True,True,False,[]
3242,"Did u even watch the full video, the indian man steped right into the crowd the boys didnt do anything , except claims harrasment after a black kid said jesus was black, dont say white bois im white and friends with tons of colored , whatchu got up rn is racism",Critique/Other,-0.018525,"[('u', []), ('man', []), ('boys', []), ('harrasment', []), ('kid', []), ('jesus', []), ('i', ['1']), ('whatchu', [])]","['do', 'said', 'black', 'white', 'friends']","[['Did', 'u', 'even', 'video'], ['watch', ',', 'man', 'into', 'do'], ['boys', 'did', 'nt', 'anything', ','], ['steped', 'except', 'harrasment', 'after', 'kid', 'was', 'is'], ['jesus', 'black'], ['i', 'white', 'and', 'friends'], ['with'], ['say', 'got', 'racism']]",True,False,False,"[('indian', 'NORP'), ('jesus', 'PERSON')]"
7226,"Bacon to keep the happy, smart",Critique/Other,0.401900,"[('Bacon', [])]",['keep'],"[['Bacon', 'to', 'smart']]",False,False,False,"[('Bacon', 'ORG')]"


# Feature explanation 

In [42]:
test = "HOE you mad? damn right our PRESI§DENT IS BLACK ,tf you gone do bouh it? he smartn than romney ass : FUCK OBAMA, dumb ass nigger"
test ="I believe Russians are beautiful and Justin Trudeau is an angel, I love him"
test = 'Bell, based in Los Angeles, makes and distributes electronic, computer and building products.'
test = "That's PRESIDENT OBAMA to you tayyoung_: FUCK OBAMA, dumb ass nigger"
instance1 = nlp(test)
    # for token in doc_text:
    #     print(token.text," : ",token.dep_, '|', token.head)
for token in instance1:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
        [child for child in token.children])

token.children

That nsubj 's AUX []
's ROOT 's AUX [That, OBAMA, to, tayyoung, _, :]
PRESIDENT compound OBAMA PROPN []
OBAMA attr 's AUX [PRESIDENT]
to prep 's AUX [you]
you pobj to ADP []
tayyoung attr 's AUX []
_ attr 's AUX []
: punct 's AUX []
FUCK compound OBAMA PROPN []
OBAMA ROOT OBAMA PROPN [FUCK, ,, nigger]
, punct OBAMA PROPN []
dumb amod nigger NOUN []
ass compound nigger NOUN []
nigger appos OBAMA PROPN [dumb, ass]


In [45]:
test = "That's PRESIDENT OBAMA to you tayyoung_: FUCK OBAMA, dumb ass nigger"
test = "HOE you mad? damn right our PRESI§DENT IS BLACK ,tf you gone do bouh it? he smartn than romney ass : FUCK OBAMA, dumb ass nigger"
test = 'Bell, based in Los Angeles, makes and distributes electronic, computer and building products.'
test ="I believe Russians are beautiful and Justin Trudeau is an angel, I love him"
instance = nlp(test)
#This part of the library is imported for graphic visualization only.
from spacy import displacy

#For our doc object, we display the dependency graph, which allows us to better understand/analyze these relationships manually
displacy.render(instance, style="dep")

In [62]:
# Test the CCOMP function 
test = "That's PRESIDENT OBAMA to you tayyoung_: FUCK OBAMA, dumb ass nigger"
test = 'Bell, based in Los Angeles, makes and distributes electronic, computer and building products.'
test = "She does not look very beautiful"
test ="I believe Russians are beautiful and Justin Trudeau is an angel, I love him"
doc = nlp(test)
# Extract adjectives, nouns, subjects and complements/conjunctions/root
adjectives = [token.text for token in doc if token.pos_ == "ADJ"]
nouns = [token.text for token in doc if token.pos_ == "NOUN"]
subjects = [(token.text, token.morph.get('Person')) for token in doc if token.dep_ == "nsubj"]
complements = [token.text for token in doc if token.dep_ in ("ccomp", "conj", "ROOT")]

# Extract children of ccomp or conj
ccomp_conj_children = []
ccomp_conj_sentiments = []
for token in doc:
    if token.dep_ in ("acomp", 'ROOT', 'ccomp'): # "ccomp", "obj", 'ROOT''AUX'
        children = [child.text for child in token.children]
        #children = token.text
        ccomp_conj_children.append(children)
        # Compute sentiment scores for each child and append to list
        #ccomp_conj_sentiments.extend([analyzer.polarity_scores(child)["compound"] for child in children])

print(ccomp_conj_children)
print(ccomp_conj_sentiments)


[['I', 'are'], ['Russians', 'beautiful', 'and', 'is'], [], ['believe', ',', 'I', 'him']]
[]
